<a href="https://colab.research.google.com/github/Adirgos/Ydata/blob/Kaggle/Adir_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

## General Notes:
- Garage columns (4 text + 1 numecric) have 1379 entries (81 NANs)
  Other 2 Garage cols has 81 zeros. 
- only numeric values missing:
  - LotFrontage: 1201
  - MasVnrArea: 1452
  - GarageYrBlt: 1379
  these are the only float columns in the dataset
- is there any logic behind missing values?
- normalization by Neighborhood?
- are missing numeric values filled with 0?
- subprime-crisis dates

## Suspicious columns:
biased volumns - columns that contain bias (few values that appear in most items):
(I made a DF from this list later on)
- MSZoning: 93% are RL or RM
- Street: 99% is Pave
- LotShape: 96% are Reg or IR1
- LandContour: 90% are Lvl
- Utilities: 99.9% are AllPub
- LotConfig: 90% are Inside or Corner
- LandSlope: 95% are Gtl
- BldgType: 91% are 1Fam or TwnhsE
- RoofStyle: 97% are Gable or Hip
- RoofMatl: 98% are CompShg
- MasVnrType: 90% are None or BrkFace
- ExterQual: 95% are TA or Gd
- ExterCond: 98% are TA or Gd
- BsmtCond: 92% are TA
- BsmtFinType2: 88% are Unf
- Heating: 97% are Gasa
- CentralAir: 93% are Y
- Electrical: 91% are SBrkr
- Functional: 93% are Typ
- GarageType: 91% are Attchd or Detchd
- GarageQual: 95% are TA
- GarageCond: 96% are TA
- PavedDrive: 92% are Y
- SaleType: 95% of values are WD or New
- SaleCondition: 98% are Normal, Partial or Abnormal
- MaxVnrArea: 60% are 0
- BsmtFinSF1: 32% are 0
- BsmtFinSF2: 86% are 0
- 2ndFlrSF: 57% are 0
- LowQualFinSF: 98% are 0
- BsmtHalfBath: 94% are 0
- KitchenAbvGr: 95% are 1
- WoodDeckSF: 52% are 0
- OpenPorchSF: 45% are 0
- EnclosedPorch: 86% are 0
- 3SsnPorch: 98% are 0
- ScreenPorch: 92% are 0
- PoolArea: 99% are 0
- MiscVal: 96% are 0



thin columns - columns with very little values:
- Alley: only 91 values
- FireplaceQu: 770 values
- PoolQC: only 7 values
- Fence: only 281 values
- MiscFeature: only 54 values



Condition 1 and Condition 2: strange relationship, most items in Norm-Norm

## Text to num:
- CentralAir: Y=1, N=0
- PavedDrive: Y=1, P=0.5, N=0

## Ideas for combinations:
- Fence & Neighborhood
- MiscVal & MiscFeature
- SaleType & missing values
- HeatingQC & neighborhood
- YearBuilt & BsmtCond
- TotRmsAbvGrd & Neighborhood
- GarageYrBlt & YearBuilt
- driveway & garage?

## Potential strong Features:
- MSSubClass
- MSZoning
- LotArea
- Utilities
- Neighborhood
- Condition1
- Condition2
- BldgType
- HouseStyle
- OverallQual
- OverallCond
- YearBuilt
- YearRemodAdd
- Exterior1st if asbest
- Exterior2nd if asbest
- BsmtQual - maybe related to foundations
- BsmtCond - maybe related to foundations
- BsmtFinType1
- BsmtFinType1
- HeatingQC
- 1stFlrSF
- 2ndFlrSF
- GrLivArea
- KitchenQual
- TotRmsAbvGrd
- Functional
- GarageType
- GarageCars
- MoSold

# Code

## Imports

In [39]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
df = pd.read_csv('train.csv')

In [ ]:
df.info()

In [3]:
data = df.loc[:, ~df.columns.isin(['SalePrice'])]
labels = df[['SalePrice']]

In [9]:
#@title data descritpion
print(open('data_description.txt', 'r').read())

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

## Text cols

In [55]:
text_cols = data.select_dtypes(include='object')
print('there are',text_cols.columns.size,'text columns')

there are 43 text columns


In [ ]:
#@title text columns' values distribution
for col in text_cols:
  actual_counts = text_cols[col].value_counts()
  normalized_values = text_cols[col].value_counts(normalize=True)
  result_df = pd.DataFrame({'Actual': actual_counts, 'Normalized': normalized_values})
  print(col, ' - ', text_cols[col].value_counts().sum(), '\n', result_df, '\n\n')

## Num cols

In [47]:
num_cols = data.select_dtypes(exclude='object')
print('there are', num_cols.columns.size, 'numeric columns')

there are 37 numeric columns


In [0]:
num_cols.info()

In [0]:
#@title numeric columns' values distribution
for col in num_cols:
  actual_counts = num_cols[col].value_counts()
  normalized_values = num_cols[col].value_counts(normalize=True)
  result_df = pd.DataFrame({'Actual': actual_counts, 'Normalized': normalized_values})
  print(col, ' - ', num_cols[col].value_counts().sum(), '\n', result_df, '\n\n')

In [0]:
#@title distribution of numerics
for column in data.select_dtypes(include='number').columns:
    plt.figure(figsize=(6, 4))
    data[column].hist()
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

In [0]:
imputed_numerics = num_cols.copy()
imputed_numerics = imputed_numerics.apply(lambda col: col.fillna(col.mean()))
imputed_numerics.info()

In [0]:
num_cols.describe().T

In [0]:
imputed_numerics.describe().T

## Biased columns

In [40]:
text = (open('Biased columns', 'r').read())

pattern = re.compile(r"(\w+): (\d+)% are (.+)")
matches = pattern.findall(text)

cols_df = pd.DataFrame(matches, columns=['feature', 'freq', 'dominant values'])

# Convert 'percents' column to numeric
cols_df['freq'] = cols_df['freq'].astype(float)

# Reorder columns
cols_df = cols_df[['feature', 'freq', 'dominant values']]

cols_df

,feature,freq,dominant values
0,MSZoning,93.0,RL or RM
1,LotShape,96.0,Reg or IR1
2,LandContour,90.0,Lvl
3,LotConfig,90.0,Inside or Corner
4,LandSlope,95.0,Gtl
5,BldgType,91.0,1Fam or TwnhsE
6,RoofStyle,97.0,Gable or Hip
7,RoofMatl,98.0,CompShg
8,MasVnrType,90.0,None or BrkFace
9,ExterQual,95.0,TA or Gd


# Playground

## Garage

In [23]:
sorted(data.columns.tolist())

['1stFlrSF',
 '2ndFlrSF',
 '3SsnPorch',
 'Alley',
 'BedroomAbvGr',
 'BldgType',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFinType1',
 'BsmtFinType2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtQual',
 'BsmtUnfSF',
 'CentralAir',
 'Condition1',
 'Condition2',
 'Electrical',
 'EnclosedPorch',
 'ExterCond',
 'ExterQual',
 'Exterior1st',
 'Exterior2nd',
 'Fence',
 'FireplaceQu',
 'Fireplaces',
 'Foundation',
 'FullBath',
 'Functional',
 'GarageArea',
 'GarageCars',
 'GarageCond',
 'GarageFinish',
 'GarageQual',
 'GarageType',
 'GarageYrBlt',
 'GrLivArea',
 'HalfBath',
 'Heating',
 'HeatingQC',
 'HouseStyle',
 'Id',
 'KitchenAbvGr',
 'KitchenQual',
 'LandContour',
 'LandSlope',
 'LotArea',
 'LotConfig',
 'LotFrontage',
 'LotShape',
 'LowQualFinSF',
 'MSSubClass',
 'MSZoning',
 'MasVnrArea',
 'MasVnrType',
 'MiscFeature',
 'MiscVal',
 'MoSold',
 'Neighborhood',
 'OpenPorchSF',
 'OverallCond',
 'OverallQual',
 'PavedDrive',
 'PoolArea',
 'PoolQC',
 'RoofMatl',
 'RoofStyle

In [39]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [32]:
col = 'YearRemodAdd'
cols = [col, 'GarageArea', 'GarageCars', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt']
data[cols]

,YearRemodAdd,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt
0,2003,548,2,TA,RFn,TA,Attchd,2003.0
1,1976,460,2,TA,RFn,TA,Attchd,1976.0
2,2002,608,2,TA,RFn,TA,Attchd,2001.0
3,1970,642,3,TA,Unf,TA,Detchd,1998.0
4,2000,836,3,TA,RFn,TA,Attchd,2000.0
...,...,...,...,...,...,...,...,...
1455,2000,460,2,TA,RFn,TA,Attchd,1999.0
1456,1988,500,2,TA,Unf,TA,Attchd,1978.0
1457,2006,252,1,TA,RFn,TA,Attchd,1941.0
1458,1996,240,1,TA,Unf,TA,Attchd,1950.0


In [28]:
1460 - 1379

81

## End